In [ ]:
import matplotlib.pyplot as plt
import numpy as np, seaborn as sns

sales_yoy_gwth_pct = 0.2
sales_yoy_gwth_pct_normal_dist = np.random.normal(loc=sales_yoy_gwth_pct, scale=0.02, size=10000)
fig, ax = plt.subplots(figsize=(15,5))
plt.title("Normal distribution for the Sales YoY Growth Rate(%)")
ax1 = sns.histplot(sales_yoy_gwth_pct_normal_dist, label='normal dist', ax=ax, kde=True, stat='density', linewidth=0)
plt.legend()
plt.show()

In [ ]:
import pandas as pd

years = [2021,2022,2023,2024,2025,2026]
df = pd.DataFrame(index=years, columns=['sales', 'int_exp', 'var_nwc'])
df.iat[0,0] = 250
df.iat[0,1] = -7.5
df.iat[0,2] = -3
print()
for i in range(1, len(years)):
    df.loc[df.index[i],'sales'] = df.loc[df.index[i - 1],'sales'] * (1 + 0.2)
    df.loc[df.index[i],'int_exp'] = df.loc[df.index[i - 1],'int_exp'] * (1 + 0.2)
    df.loc[df.index[i],'var_nwc'] = df.loc[df.index[i - 1],'var_nwc'] * (1 + 0.2)

df['cogs'] = -df['sales'] * 0.63
df['sga'] = -df['sales'] * 0.15
df['dep_amort'] = -df['sales'] * 0.015
df['profit_bef_tax'] = (df['sales'] + df['cogs'] + df['sga'] + df['dep_amort'] + df['int_exp'])
df['tax'] = -df['profit_bef_tax'] * 0.33
df['net_income'] = df['profit_bef_tax'] + df['tax']
df['capex'] = df['dep_amort'] * 1
df['fcf'] = df['net_income'] - df['int_exp'] - df['dep_amort'] + df['var_nwc'] + df['capex']

wacc = 0.09
perp_gwth_rate = 0.015
dfc_factor = [1/(1 + wacc) ** x for x in range(1, len(years))]
term_value = (df['fcf'].iloc[-1] * (1 + perp_gwth_rate) / (wacc - perp_gwth_rate)) * dfc_factor[-1]

npv = df['fcf'][1:] * dfc_factor
comp_value = npv.sum() + term_value
comp_value